# Recommender Playlists

Exploration implementing recommender system techniques using Spotify playlist data:

- Popularity Recommender: Recommend popular songs regardless of user's preferences
- Content-based Recommender: Use song attributes (e.g. genre) to recommend similar songs
- Collaborative Recommender: Predict what songs a user might be interested in based on a collection of preference information from multiple users
- Hybrid Recommender: A hybrid approach generally outperforms a single model and can be used to overcome some of the common problems in recommender systems such as the cold start problem and the sparsity problem
- HybridPopularity Recommender: An extension of the hybrid approach which applies weighting/mixes in songs based on popularity

For the purposes of this dataset we will focus on the playlist_tracks_df dataset and treat different playlists as different users. This is a strong approach for exploring recommendation techniques but leads to a problem the best way to recommend songs for our 'users' is based on genre (as my playlists tend to be genre-based).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import yaml

In [2]:
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, normalize

In [3]:
# To create a playlist and add tracks
import spotipy 
from spotipy.oauth2 import SpotifyOAuth

## Import Data

- Artist and track data was pulled using the Spotify API via the spotipy package
- Data was saved in pickle format using music_data.py and data_functions.py modules
- The data can now be quickly read by multiple workflows

In [4]:
top_artist_df = pd.read_pickle("spotify/top_artists.pkl")
followed_artists_df = pd.read_pickle("spotify/followed_artists.pkl")
top_tracks_df = pd.read_pickle("spotify/top_tracks.pkl")
saved_tracks_df = pd.read_pickle("spotify/saved_tracks.pkl")
playlist_tracks_df = pd.read_pickle("spotify/playlist_tracks.pkl")
recommendation_tracks_df = pd.read_pickle("spotify/recommendation_tracks.pkl")

In [5]:
playlist_tracks_df['popularity'] = playlist_tracks_df['popularity'] / 100  # normalise popularity feature between 0 and 1
playlist_tracks_df.head()

,id,name,popularity,type,is_local,explicit,duration_ms,disc_number,track_number,artist_id,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,time_signature
0,6fgbQt13JlpN59PytgTMsA,Snow (Hey Oh),0.61,audio_features,False,False,334667,1,1,0L8ExT028jH3ddEcZwqJJ5,...,0.0499,0.116000,0.000017,0.119,0.599,104.655,spotify:track:6fgbQt13JlpN59PytgTMsA,https://api.spotify.com/v1/tracks/6fgbQt13JlpN...,https://api.spotify.com/v1/audio-analysis/6fgb...,4
1,3JOdLCIBzQYwHIvpN3isVf,Grand Theft Autumn / Where Is Your Boy,0.62,audio_features,False,False,191373,1,3,4UXqAaa6dQYAk18Lv7PEgX,...,0.0608,0.000706,0.000000,0.275,0.720,135.450,spotify:track:3JOdLCIBzQYwHIvpN3isVf,https://api.spotify.com/v1/tracks/3JOdLCIBzQYw...,https://api.spotify.com/v1/audio-analysis/3JOd...,4
2,7pAT4dOUzjq8Ziap5ShIqC,Where'd You Go (feat. Holly Brook & Jonah Matr...,0.58,audio_features,False,True,231867,1,6,7dWYWUbO68rXJOcyA7SpJk,...,0.2380,0.262000,0.001970,0.113,0.250,179.999,spotify:track:7pAT4dOUzjq8Ziap5ShIqC,https://api.spotify.com/v1/tracks/7pAT4dOUzjq8...,https://api.spotify.com/v1/audio-analysis/7pAT...,4
3,1b7vg5T9YKR3NNqXfBYRF7,Check Yes Juliet,0.53,audio_features,False,False,220133,1,3,3ao3jf5d70Tf4fPh2bnXVl,...,0.0774,0.002400,0.000000,0.163,0.314,166.866,spotify:track:1b7vg5T9YKR3NNqXfBYRF7,https://api.spotify.com/v1/tracks/1b7vg5T9YKR3...,https://api.spotify.com/v1/audio-analysis/1b7v...,4
4,12qZHAeOyTf93YAWvGDTat,All The Small Things,0.00,audio_features,False,False,168000,1,8,6FBDaR13swtiWwGhX1WQsP,...,0.0505,0.008440,0.000000,0.529,0.712,148.119,spotify:track:12qZHAeOyTf93YAWvGDTat,https://api.spotify.com/v1/tracks/12qZHAeOyTf9...,https://api.spotify.com/v1/audio-analysis/12qZ...,4


In [6]:
with open("spotify/playlists.yml", 'r') as stream:
    playlist_ids = yaml.safe_load(stream)

## Evaluation Metric

Here we use the Top-N accuracy metric, which evaluates the accuracy of the top recommendations provided to a user by comparing to the items the user has actually interacted in test set. This evaluation method works as follows:

- For each user
    - For each item the user has interacted in test set
        - Sample n other items the user has never interacted with (assume these are not relevant, but the user may just have not been aware of them)
        - Ask the recommender model to produce a ranked list of recommended items, from a set composed one interacted item and the 100 non-interacted ("non-relevant") items
        - Compute the Top-N accuracy metrics for this user and interacted item from the recommendations ranked list (is the item along the Top-N ranked items)
- Aggregate the global Top-N accuracy metrics

In [7]:
class ModelEvaluator:
    
    def __init__(self, tracks):
        self.tracks = tracks
        
    def evaluate_model_for_playlist(self, model, playlist_id, n=100, seed=42):
        # Getting the items in test set
        tracks_interacted, tracks_not_interacted = get_interacted_tracks(self.tracks, playlist_id)
        train, test = train_test_split(tracks_interacted, test_size=0.2, random_state=seed)
        # Getting a ranked recommendation list from a model for a given user
        ranked_recommendations_df = model.recommend_tracks(playlist_id)

        hits_at_5_count, hits_at_10_count = 0, 0
        for index, row in test.iterrows():
            non_interacted_sample = tracks_not_interacted.sample(n, random_state=seed)
            evaluation_ids = [row['id']] + non_interacted_sample['id'].tolist()
            evaluation_recommendations_df = ranked_recommendations_df[ranked_recommendations_df['id'].isin(evaluation_ids)]
            # Verifying if the current interacted item is among the Top-N recommended items
            hits_at_5_count += 1 if row['id'] in evaluation_recommendations_df['id'][:5].tolist() else 0
            hits_at_10_count += 1 if row['id'] in evaluation_recommendations_df['id'][:10].tolist() else 0

        playlist_metrics = {'n': n,
                            'evaluation_count': len(test),
                            'hits@5': hits_at_5_count,
                            'hits@10': hits_at_10_count, 
                            'recall@5': hits_at_5_count / len(test),
                            'recall@10': hits_at_10_count / len(test),
                           }
        
        return playlist_metrics

    def evaluate_model(self, model, n=100, seed=42):
        playlists = []
        for playlist_id in self.tracks['playlist_id'].unique():
            playlist_metrics = self.evaluate_model_for_playlist(model, playlist_id, n=n, seed=seed)  
            playlist_metrics['playlist_id'] = playlist_id
            playlists.append(playlist_metrics)

        detailed_playlists_metrics = pd.DataFrame(playlists).sort_values('evaluation_count', ascending=False)
        
        global_recall_at_5 = detailed_playlists_metrics['hits@5'].sum() / detailed_playlists_metrics['evaluation_count'].sum()
        global_recall_at_10 = detailed_playlists_metrics['hits@10'].sum() / detailed_playlists_metrics['evaluation_count'].sum()
        
        global_metrics = {'model_name': model.model_name,
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10,
                         }  
                            
        return global_metrics, detailed_playlists_metrics
    
model_evaluator = ModelEvaluator(playlist_tracks_df)

### Interacted tracks

Now to evaluate a model for a playlist (and overall), we need to get both iteracted and non-interacted tracks for a playlist.

In [8]:
def get_interacted_tracks(tracks, playlist_id, drop_duplicates=True):
    interacted_track_ids = set(tracks[tracks['playlist_id'] == playlist_id]['id'])
    tracks_interacted = tracks[tracks['id'].isin(interacted_track_ids)]
    tracks_not_interacted = tracks[~tracks['id'].isin(interacted_track_ids)]

    if drop_duplicates is True:
        tracks_interacted = tracks_interacted.drop_duplicates(subset='id', keep="first").reset_index()
        tracks_not_interacted = tracks_not_interacted.drop_duplicates(subset='id', keep="first").reset_index()

    return tracks_interacted, tracks_not_interacted

In [9]:
interacted_tracks, non_interacted_tracks = get_interacted_tracks(playlist_tracks_df, playlist_ids['Chill'])

## Popularity Recommender

A popularity based recommender recommends songs in order of overall popularity, regardless of what the user has listened to. Spotify's 'audio features' API call automatically comes with a 'popularity' feature. Although it is 0 in ~10% of cases (higher than expectrf - these are probably default null values), this is perfect for creating a Popularity recommender.

As song popularity generally accounts for the "wisdom of the crowds", it usually provides good recommendations overall. However this isn't tailored to the user in particular, as a good recommender system should be.

In [10]:
class PopularityRecommender:
    
    def __init__(self, tracks):
        self.tracks = tracks
        self.model_name = 'Popularity Recommender'
    
    def recommend_tracks(self, playlist_id, ignore_ids=[]):
        recommendations_df = self.tracks[~self.tracks['id'].isin(ignore_ids)] \
                                .drop_duplicates(subset='id', keep="first").reset_index() \
                                .sort_values('popularity', ascending=False)

        return recommendations_df
    
popularity_model = PopularityRecommender(playlist_tracks_df)

In [11]:
# You can see this is essentially sorted by popularity
popularity_model_recommendations = popularity_model.recommend_tracks(playlist_ids['Chill'], interacted_tracks['id'].tolist())
popularity_model_recommendations[['id', 'name', 'artist_name', 'album_name', 'popularity']].head()

,id,name,artist_name,album_name,popularity
203,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,The Weeknd,After Hours,0.96
301,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",0.88
406,3AJwUDP919kvQ9QcozQPxg,Yellow,Coldplay,Parachutes,0.86
292,2dpaYNEQHiRxtZbfNsse99,Happier,Marshmello,Happier,0.85
2304,7w87IxuO7BDcJ3YUqCyMTT,Pumped Up Kicks,Foster The People,Torches,0.85


In [12]:
popularity_model_metrics, popularity_model_details = model_evaluator.evaluate_model(popularity_model)

print(popularity_model_metrics)
popularity_model_details[[x for x in popularity_model_details.columns if x != 'playlist_id']] \
    .sort_values('recall@5', ascending=False) \
    .head(10)

{'model_name': 'Popularity Recommender', 'recall@5': 0.09318497913769123, 'recall@10': 0.17385257301808066}


,n,evaluation_count,hits@5,hits@10,recall@5,recall@10
28,100,2,2,2,1.000000,1.000000
2,100,14,7,7,0.500000,0.500000
12,100,17,7,9,0.411765,0.529412
9,100,17,7,9,0.411765,0.529412
14,100,20,8,11,0.400000,0.550000
41,100,21,8,11,0.380952,0.523810
0,100,19,6,6,0.315789,0.315789
11,100,36,9,11,0.250000,0.305556
46,100,4,1,1,0.250000,0.250000
15,100,20,5,6,0.250000,0.300000


As one would expect, solely recommending by popularity is a poor way to recommend tracks. As we will see later it is however a good method to mix in for variety and to avoid the cold-start problem.

## Content-based Recommender

A content-based recommender leverages attributes from items the user has interacted with to recommend similar items. As it depends only on the past this method avoids the cold-start problem for implementation.

For text items we can use a popular information retrieval method used in search engines named TF-IDF. This technique converts unstructured text into a vector structure, where each word is represented by a position in the vector, and the value measures how relevant a given word is for an article. We can then compute the cosine similarity between the user vector and the initial sparse matrix (all users).

### TF-IDF

First we need to apply the TF-IDF technique, and use it to build playlist profiles

In [13]:
def get_tfidf(tracks, ngram_range=(1,2), min_df=0.003, max_df=0.5, max_features=5000):
    # Transform list cols to string, we use bigrams later so no need to remove spaces
    tracks['genres_str'] = tracks['genres'].apply(lambda x: ' '.join(x))

    # Vector size 5000 model
    vectorizer = TfidfVectorizer(analyzer='word',
                                 ngram_range=ngram_range,
                                 min_df=min_df,
                                 max_df=max_df,
                                 max_features=max_features,
                                 stop_words=stopwords.words('english'))  # might need to download stopwords, follow prompt

    # Don't include album_genres column here as similar to genres column and we don't want additional genre weighting in this case 
    # vectorizer.fit_transform takes a string
    tfidf_matrix = vectorizer.fit_transform(tracks['name'] + ' ' +
                                            tracks['artist_name'] + ' ' +
                                            tracks['album_name'] + ' ' +
                                            tracks['playlist_name'] + ' ' +
                                            tracks['genres_str']
                                           )
    tfidf_feature_names = vectorizer.get_feature_names()

    return tfidf_matrix, tfidf_feature_names

In [14]:
tfidf_matrix, tfidf_feature_names = get_tfidf(playlist_tracks_df)
tfidf_matrix

<3505x948 sparse matrix of type '<class 'numpy.float64'>'
	with 47341 stored elements in Compressed Sparse Row format>

In [15]:
def get_track_profile(tracks, track_id, tfidf_matrix):
    idx = tracks['id'].tolist().index(track_id)
    track_profile = tfidf_matrix[idx:idx+1]
    return track_profile

def get_track_profiles(tracks, track_ids, tfidf_matrix):
    track_profiles_list = [get_track_profile(tracks, x, tfidf_matrix) for x in track_ids]
    track_profiles = vstack(track_profiles_list)
    return track_profiles

def build_playlists_profile(tracks, playlist_id, interactions_indexed_df, tfidf_matrix):
    # There isn't any weighting we want to do in this case, 
    # but a common approach is weighting by interaction strength (liking, commenting, etc.)
    interaction_tracks_df = interactions_indexed_df.loc[playlist_id]  # duplicate interacted tracks, filter to current playlist
    playlist_track_profiles = get_track_profiles(tracks, interaction_tracks_df['id'], tfidf_matrix)
    # Weighted average for event strengths
#     playlist_track_strengths = np.array(interaction_tracks_df['event_strength']).reshape(-1,1)
#     playlist_track_profiles_array = np.sum(playlist_track_profiles.multiply(playlist_track_strengths), axis=0) / np.sum(playlist_track_strengths)
    playlist_track_profiles_array = np.sum(playlist_track_profiles, axis=0)  # Flattens no_tracksx948 matrix to 1x948 array
    playlist_track_profiles_norm = normalize(playlist_track_profiles_array)
    return playlist_track_profiles_norm

def build_playlists_profiles(tracks, tfidf_matrix): 
    playlist_profiles = {}
    for playlist_id in tracks['playlist_id'].unique():
        interacted_tracks, non_interacted_tracks = get_interacted_tracks(tracks, playlist_id, drop_duplicates=False)
        playlist_profiles[playlist_id] = build_playlists_profile(tracks, playlist_id, interacted_tracks.set_index('playlist_id'), tfidf_matrix)
    return playlist_profiles

In [16]:
playlist_profiles = build_playlists_profiles(playlist_tracks_df, tfidf_matrix)
len(playlist_profiles), len(playlist_tracks_df['playlist_id'].unique())  # all playlists accounted for

(49, 49)

In [17]:
# Get the keywords for my "Chill" playlist
chill_profile = playlist_profiles[playlist_ids['Chill']]
print(chill_profile.shape)  # 95 songs of vector length 948
pd.DataFrame(sorted(zip(tfidf_feature_names, chill_profile.flatten().tolist()), key=lambda x: -x[1])[:10],  # sort by value desc
             columns=['token', 'relevance'])

(1, 948)


,token,relevance
0,chill,0.515348
1,rock,0.315593
2,punk,0.208226
3,dance,0.192835
4,pop punk,0.186408
5,modern,0.166279
6,chill pop,0.148753
7,rap,0.147725
8,chill modern,0.141358
9,modern rock,0.137031


### Apply Content-based Recommender

Now with our playlist profiles setup, we can apply a content-based recommender.

In [18]:
class ContentRecommender:
    
    def __init__(self, tracks, tfidf_matrix, playlist_profiles):
        self.tracks = tracks
        self.tfidf_matrix = tfidf_matrix
        self.playlist_profiles = playlist_profiles
        self.model_name = 'Content-based Recommender'

    def _get_similar_tracks(self, playlist_id):
        #Computes the cosine similarity between the playlist profile and all profiles
        cosine_similarities = cosine_similarity(self.playlist_profiles[playlist_id], self.tfidf_matrix)
        similar_indices = cosine_similarities.argsort().flatten()
        #Sort the similar tracks by similarity
        similar_tracks = sorted([(self.tracks['id'].tolist()[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_tracks
        
    def recommend_tracks(self, playlist_id, ignore_ids=[]):
        similar_tracks = self._get_similar_tracks(playlist_id)
        similar_tracks_non_interacted = list(filter(lambda x: x[0] not in ignore_ids, similar_tracks))
        recommendations_df = pd.DataFrame(similar_tracks_non_interacted, columns=['id', 'recStrength']) \
                                .drop_duplicates(subset='id', keep="first").reset_index() \
                                .sort_values('recStrength', ascending=False)

        return recommendations_df
    
content_model = ContentRecommender(playlist_tracks_df, tfidf_matrix, playlist_profiles)

In [19]:
content_model_recommendations = content_model.recommend_tracks(playlist_ids['Chill'], interacted_tracks['id'].tolist())
# Get track details from original track data
content_model_recommendations_name = pd.merge(content_model_recommendations, playlist_tracks_df.drop_duplicates(subset='id', keep="first"), how='left', on='id')
content_model_recommendations_name[['id', 'name', 'artist_name', 'album_name', 'recStrength']].head()

,id,name,artist_name,album_name,recStrength
0,6QZ8h3RqIgTRTo3hfaqryx,Black Butterflies and Déjà Vu,The Maine,Lovely Little Lonely,0.481373
1,7DUCNQRudZwc5TzxThlaRg,Rescue Me,You Me At Six,Rescue Me,0.473033
2,4GMCZUi6cVQJ5wPWwJ36rW,Underdog,You Me At Six,Hold Me Down,0.473033
3,0J8KmsNtnDY0FdfCD1i2zx,Straight to My Head,You Me At Six,VI,0.473033
4,2AeaghRmCn2AXapvOKMbW4,Cold Night,You Me At Six,Cavalier Youth,0.465654


In [20]:
content_model_metrics, content_model_details = model_evaluator.evaluate_model(content_model)

print(content_model_metrics)
content_model_details[[x for x in content_model_details.columns if x != 'playlist_id']] \
    .sort_values('recall@5', ascending=False) \
    .head()

{'model_name': 'Content-based Recommender', 'recall@5': 0.9123783031988874, 'recall@10': 0.972183588317107}


,n,evaluation_count,hits@5,hits@10,recall@5,recall@10
4,100,1,1,1,1.0,1.0
20,100,3,3,3,1.0,1.0
34,100,2,2,2,1.0,1.0
6,100,11,11,11,1.0,1.0
26,100,55,55,55,1.0,1.0


From the token relevance and exceptionally high recall it like the genre of a song is a very powerful recommender for me personally. It is likely that the Spotify breakdown of genres e.g. modern pop and chill pop helps a lot with this, alongside the bias of the dataset being playlist based (which tend to be genre/mood based).

## Collaborative Recommender

There are two main implementation strategies for a collaborative recommender:

- **Memory-based:** Computes user (user-based) or item (item-based) similarities based on past user interactions with items
- **Model-based:** Like in the accompanying recommender_playlists.ipynb file, models are developed using different ML algorithms such as neural networks, bayesian networks, and clustering models, as well as latent factor models such as Singular Value Decomposition (SVD) and, probabilistic latent semantic analysis

### Matrix Factorisation

Latent factor models compress a user-item matrix into a low-dimensional representation in terms of latent factors. This has several advantages including fewer missing values, better scalability during similarity comparison, and that we will be dealing with a much smaller matrix in lower-dimensional space. Note that model-based approaches handle the sparsity of the original matrix better than memory-based ones. An important number is the number of selected factors to factor the user-item matrix. The higher the number of factors, the more precise is the factorisation, and the more details of the original matrix which are memorized. Reducing the number of factors increases the model generalisation whilst too many factors can lead to overfitting (high variance).

Here we a use popular latent factor model named Singular Value Decomposition (SVD) which is available in SciPy. Other choices might have been surprise, mrec, or python-recsys.

In [21]:
#Creating a sparse pivot table with users in rows and items in columns
playlist_tracks_df['event_strength'] = 1  ## create dummy column for pivot value
playlist_tracks_matrix_df = playlist_tracks_df.pivot_table(index='playlist_id',
                                                           columns='id',
                                                           values='event_strength',
                                                           aggfunc='sum',
                                                          ).fillna(0)

playlist_tracks_matrix_df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
playlist_tracks_matrix = playlist_tracks_matrix_df.values
playlist_tracks_sparse = csr_matrix(playlist_tracks_matrix)
playlist_tracks_sparse

<49x2609 sparse matrix of type '<class 'numpy.float64'>'
	with 3499 stored elements in Compressed Sparse Row format>

In [23]:
# Performs matrix factorisation of the original user item matrix
u, s, vt = svds(playlist_tracks_sparse, k = 15)  # k is number of factors
s = np.diag(s)
print(u.shape, vt.shape, s.shape)

(49, 15) (15, 2609) (15, 15)


In [24]:
playlist_predicted_ratings = np.dot(np.dot(u, s), vt) 
playlist_predicted_ratings
# MinMaxScaler doesn't work for global min(), max()
playlist_predicted_ratings_norm = (playlist_predicted_ratings - playlist_predicted_ratings.min()
                                  ) / (playlist_predicted_ratings.max() - playlist_predicted_ratings.min())
playlist_predicted_ratings_norm

array([[0.17640704, 0.17640294, 0.17647311, ..., 0.17725056, 0.17637331,
        0.17749934],
       [0.1748612 , 0.17509977, 0.15775461, ..., 0.17791478, 0.54944338,
        0.17798536],
       [0.18397378, 0.22086351, 0.64397648, ..., 0.20251616, 0.19243753,
        0.14780581],
       ...,
       [0.1763876 , 0.1763876 , 0.1763876 , ..., 0.1763876 , 0.1763876 ,
        0.1763876 ],
       [0.1717029 , 0.180459  , 0.18870632, ..., 0.25506078, 0.17570142,
        0.16681618],
       [0.17639438, 0.17623027, 0.17602779, ..., 0.17667016, 0.17641338,
        0.1770732 ]])

In [25]:
#Converting the reconstructed matrix back to a Pandas dataframe
matrix_preds_df = pd.DataFrame(playlist_predicted_ratings_norm, columns = playlist_tracks_matrix_df.columns, 
                               index=playlist_tracks_df['playlist_id'].unique()).transpose()
matrix_preds_df.shape

(2609, 49)

### Apply Collaborative Recommender

Now that we have completed our matrix factorisation, we can apply a collaborative recommender.

In [26]:
class CollaborativeRecommender:
    
    def __init__(self, tracks, matrix_preds_df):
        self.tracks = tracks
        self.matrix_preds_df = matrix_preds_df
        self.model_name = 'Collaborative Recommender'
    
    def recommend_tracks(self, playlist_id, ignore_ids=[]):
        sorted_playlist_predictions = self.matrix_preds_df[playlist_id].sort_values(ascending=False) \
                                        .reset_index().rename(columns={playlist_id: 'recStrength'})        
        recommendations_df = sorted_playlist_predictions[~sorted_playlist_predictions['id'].isin(ignore_ids)] \
                                .drop_duplicates(subset='id', keep="first").reset_index() \
                                .sort_values('recStrength', ascending = False)
        
        return recommendations_df

collaborative_model = CollaborativeRecommender(playlist_tracks_df, matrix_preds_df)

In [27]:
collaborative_model_recommendations = collaborative_model.recommend_tracks(playlist_ids['Chill'], interacted_tracks['id'].tolist())
# Get track details from original track data
collaborative_model_recommendations_name = pd.merge(collaborative_model_recommendations, playlist_tracks_df.drop_duplicates(subset='id', keep="first"), how='left', on='id')
collaborative_model_recommendations_name[['id', 'name', 'artist_name', 'album_name', 'recStrength']].head()

,id,name,artist_name,album_name,recStrength
0,58z37Jln7nsv1uMvZ8EVw2,Late Love Letter,SUNNYSIDEMJ,Late Love Letter,0.177554
1,5a9JzRCWSMe65xyBQLDQMO,Player,Dok2,The Player Pt. 1 (Original Television Soundtrack),0.177553
2,0btaXRg5MasjKksdxNMNHs,Social Distance 16,Epik High,Epik High Is Here 上 (Part 1),0.177549
3,0xKthdfyD9BgnNVVmh2fIx,수상소감 (Acceptance Speech),Epik High,Epik High Is Here 上 (Part 1),0.177549
4,31LHplV8BL8F0Uu9YbvTAY,Rosario,Epik High,Epik High Is Here 上 (Part 1),0.177549


In [28]:
collaborative_model_metrics, collaborative_model_details = model_evaluator.evaluate_model(collaborative_model)

print(collaborative_model_metrics)
collaborative_model_details[[x for x in collaborative_model_details.columns if x != 'playlist_id']] \
    .sort_values('recall@5', ascending=False) \
    .head()

{'model_name': 'Collaborative Recommender', 'recall@5': 0.23783031988873435, 'recall@10': 0.30737134909596664}


,n,evaluation_count,hits@5,hits@10,recall@5,recall@10
31,100,83,83,83,1.000000,1.000000
15,100,20,20,20,1.000000,1.000000
2,100,14,14,14,1.000000,1.000000
21,100,17,16,16,0.941176,0.941176
34,100,2,1,1,0.500000,0.500000


The collaborative recommender performs better than the popularity approach but is not as good as the content-based approach as genre appears to be the most powerful feature.

## Hybrid Recommender

Combining the content-based and collaborative approaches into a hybrid method have been shown to perform better in many studies. Here we build an ensemble which takes the weighted average of the normalized recommendation scores. As the content-based approach has a higher recall we will apply a weighting.

In [29]:
class HybridRecommender:

    def __init__(self, tracks, content_model, collaborative_model, content_weight=2, collaborative_weight=1):
        self.tracks = tracks
        self.model_name = 'Hybrid Recommender'
        self.content_model = content_model
        self.collaborative_model = collaborative_model
        # Relative weights
        self.content_weight = content_weight
        self.collaborative_weight = collaborative_weight
        
    def recommend_tracks(self, playlist_id, ignore_ids=[]): 
        
        content_recs_df = self.content_model.recommend_tracks(
            playlist_id, ignore_ids).rename(columns={'recStrength': 'recStrengthContent'})
        collaborative_recs_df = self.collaborative_model.recommend_tracks(
            playlist_id, ignore_ids).rename(columns={'recStrength': 'recStrengthCollaborative'})
        combined_recs_df = content_recs_df.merge(collaborative_recs_df,
                                                 how = 'outer', 
                                                 on = 'id',
                                                ).fillna(0)
        # Compute hybrid score based on weights
        combined_recs_df['recStrengthHybrid'] = (combined_recs_df['recStrengthContent'] * self.content_weight) \
                                                + (combined_recs_df['recStrengthCollaborative'] * self.collaborative_weight)
        recommendations_df = combined_recs_df \
                                .drop_duplicates(subset='id', keep="first").reset_index() \
                                .sort_values('recStrengthHybrid', ascending=False)

        return recommendations_df
    
hybrid_model = HybridRecommender(playlist_tracks_df, content_model, collaborative_model)

In [30]:
hybrid_model_recommendations = hybrid_model.recommend_tracks(playlist_ids['Chill'], interacted_tracks['id'].tolist())
# Get track details from original track data
hybrid_model_recommendations_name = pd.merge(hybrid_model_recommendations, playlist_tracks_df.drop_duplicates(subset='id', keep="first"), how='left', on='id')
hybrid_model_recommendations_name[['id', 'name', 'artist_name', 'album_name', 'recStrengthHybrid']].head()

,id,name,artist_name,album_name,recStrengthHybrid
0,6QZ8h3RqIgTRTo3hfaqryx,Black Butterflies and Déjà Vu,The Maine,Lovely Little Lonely,1.139124
1,7DUCNQRudZwc5TzxThlaRg,Rescue Me,You Me At Six,Rescue Me,1.122533
2,4GMCZUi6cVQJ5wPWwJ36rW,Underdog,You Me At Six,Hold Me Down,1.122459
3,0J8KmsNtnDY0FdfCD1i2zx,Straight to My Head,You Me At Six,VI,1.122459
4,2AeaghRmCn2AXapvOKMbW4,Cold Night,You Me At Six,Cavalier Youth,1.107711


In [31]:
hybrid_model_metrics, hybrid_model_details = model_evaluator.evaluate_model(hybrid_model)

print(hybrid_model_metrics)
hybrid_model_details[[x for x in hybrid_model_details.columns if x != 'playlist_id']] \
    .sort_values('recall@5', ascending=False) \
    .head()

{'model_name': 'Hybrid Recommender', 'recall@5': 0.9068150208623088, 'recall@10': 0.9666203059805285}


,n,evaluation_count,hits@5,hits@10,recall@5,recall@10
4,100,1,1,1,1.0,1.0
7,100,6,6,6,1.0,1.0
34,100,2,2,2,1.0,1.0
6,100,11,11,11,1.0,1.0
26,100,55,55,55,1.0,1.0


In this case solely applying the content-based recommender actually performs better. As discussed previously, this is because this is a playlist based dataset where each playlist tends to be genre/mood based.

### HybridPopularity Recommender

In practice we don't want to overfit on any one approach and as the content-based approach has shown a strong preference for genre for this dataset it's better to incorporate variety. Things that are popular are liked by a lot of people so we should also recommend a few things that are popular. This helps to avoid a recommendation loop where a user is consistently recommended songs from a certain genre (as similar tracks via collaborative content-based recommendations will likely be of a few set genres).

In [32]:
class HybridPopularityRecommender:

    def __init__(self, tracks, content_model, collaborative_model, popularity_model, 
                 content_weight=1, collaborative_weight=2, popularity_weight=1):
        self.tracks = tracks
        self.model_name = 'HybridPopularity Recommender'
        self.content_model = content_model
        self.collaborative_model = collaborative_model
        self.popularity_model = popularity_model
        # Relative weights
        self.content_weight = content_weight
        self.collaborative_weight = collaborative_weight
        self.popularity_weight = popularity_weight
        
    def recommend_tracks(self, playlist_id, ignore_ids=[]): 
        
        content_recs_df = self.content_model.recommend_tracks(
            playlist_id, ignore_ids).rename(columns={'recStrength': 'recStrengthContent'})
        collaborative_recs_df = self.collaborative_model.recommend_tracks(
            playlist_id, ignore_ids).rename(columns={'recStrength': 'recStrengthCollaborative'})
        popularity_recs_df = self.popularity_model.recommend_tracks(
            playlist_id, ignore_ids).rename(columns={'popularity': 'recStrengthPopularity'})
        combined_recs_df = content_recs_df.merge(collaborative_recs_df,
                                                 how = 'outer', 
                                                 on = 'id',
                                                ).merge(popularity_recs_df,
                                                        how = 'outer', 
                                                        on = 'id',
                                                       ).fillna(0)
        # Compute hybrid score based on weights
        combined_recs_df['recStrengthHybridPopularity'] = (combined_recs_df['recStrengthContent'] * self.content_weight) \
                                                            + (combined_recs_df['recStrengthCollaborative'] * self.collaborative_weight) \
                                                            + (popularity_recs_df['recStrengthPopularity'] * self.popularity_weight)
        recommendations_df = combined_recs_df \
                                .drop_duplicates(subset='id', keep="first").reset_index() \
                                .sort_values('recStrengthHybridPopularity', ascending=False)

        return recommendations_df
    
hybridpopularity_model = HybridPopularityRecommender(playlist_tracks_df, content_model, collaborative_model, popularity_model)

In [33]:
hybridpopularity_model_recommendations = hybridpopularity_model.recommend_tracks(playlist_ids['Chill'], interacted_tracks['id'].tolist())
hybridpopularity_model_recommendations[['id', 'name', 'artist_name', 'album_name', 'recStrengthHybridPopularity']].head()

,id,name,artist_name,album_name,recStrengthHybridPopularity
2,4GMCZUi6cVQJ5wPWwJ36rW,Underdog,You Me At Six,Hold Me Down,1.645819
203,2VSbEXqs6NbNiZSTcHlIDR,My Sacrifice,Creed,Weathered,1.566369
0,6QZ8h3RqIgTRTo3hfaqryx,Black Butterflies and Déjà Vu,The Maine,Lovely Little Lonely,1.554129
30,0FNPI9jhoho8rZL1J7WK1v,Sleeping In,All Time Low,Sleeping In/Some Kind Of Disaster,1.527836
11,6eDtRpS3G4UNQkaDuvQX6u,Underdog - Acoustic Version,You Me At Six,Underdog,1.522162


In [34]:
hybridpopularity_model_metrics, hybridpopularity_model_details = model_evaluator.evaluate_model(hybridpopularity_model)

print(hybridpopularity_model_metrics)
hybridpopularity_model_details[[x for x in hybridpopularity_model_details.columns if x != 'playlist_id']] \
    .sort_values('recall@5', ascending=False) \
    .head()

{'model_name': 'HybridPopularity Recommender', 'recall@5': 0.6244784422809457, 'recall@10': 0.7343532684283728}


,n,evaluation_count,hits@5,hits@10,recall@5,recall@10
4,100,1,1,1,1.0,1.0
28,100,2,2,2,1.0,1.0
35,100,4,4,4,1.0,1.0
37,100,3,3,3,1.0,1.0
2,100,14,14,14,1.0,1.0


The model weighting were set ad-hoc and should be adjusted freely. Personally I believe this HybridPopularity recommender gives the recommendations so far in practice, even though it has a lower recall than the hybrid and content-based recommender. It may therefore be better to incorporate other evaluation metrics such as one that measures variety, or increase the scope of this dataset as its current playlist based approach means that pure genre-based recommendations perform the best.

## Recommendations
- Let's see what songs the HybridPopularity recommender suggests to add to the 'Chill' playlist

In [35]:
# 243 tracks where prob_ratings >= 1.15 which is a good number
tracks_to_add = hybridpopularity_model_recommendations[hybridpopularity_model_recommendations['recStrengthHybridPopularity'] >= 1.15]['id']
len(tracks_to_add)

243

In [36]:
# Spotify API
with open("spotify/spotify_details.yml", 'r') as stream:
    spotify_details = yaml.safe_load(stream)

scope = "playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=spotify_details['client_id'],
    client_secret=spotify_details['client_secret'],
    redirect_uri=spotify_details['redirect_uri'],
    scope=scope,
))

# Create a new playlist for tracks to add - you may also add these tracks to your source playlist and proceed
new_playlist = sp.user_playlist_create(user=spotify_details['user'], 
                                       name="spotify-recommender-systems",
                                       public=False, 
                                       collaborative=False, 
                                       description="Created by https://github.com/anthonyli358/spotify-recommender-systems",
                                      )

# Add tracks to the new playlist
for id in tracks_to_add:
    sp.user_playlist_add_tracks(user=spotify_details['user'], 
                                playlist_id=new_playlist['id'], 
                                tracks=[id],
                               );

This worked very well, the playlist is lit.